<a id=“title_ID”></a>
# JWST Pipeline Validation Testing Notebook: Spec2, srctype, MIRI
Instruments Affected: MIRI

## Introduction and summary of test being run

This notebook processes an image through calwebb_spec2 and examines the output of the flat_field step. The steps are as follow:

1) Set up data path and directory and image file names.

2) Set the appropriate header keywords to test.

2) Run data through the source_type step in calwebb_spec2.

3) Compare input to output SRCTYPE keyword

## Documentation
The pipeline documentation can be found here: https://jwst-pipeline.readthedocs.io/en/latest/

The pipeline code is available on GitHub: https://github.com/spacetelescope/jwst

##Data used

The data used in this test is a simulated MIRI image created using MIRISim. The documentation for MIRISim can be found here: https://wiki.miricle.org/bin/view/Public/MIRISim_Public?

Author: Mike Engesser

## Create Temporary Directory

In [1]:
# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
data_dir = TemporaryDirectory()
os.chdir(data_dir.name)
print(data_dir)

<TemporaryDirectory '/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpm8cj6uru'>


In [2]:
import os
if 'CRDS_CACHE_TYPE' in os.environ:
    if os.environ['CRDS_CACHE_TYPE'] == 'local':
        os.environ['CRDS_PATH'] = os.path.join(os.environ['HOME'], 'crds', 'cache')
    elif os.path.isdir(os.environ['CRDS_CACHE_TYPE']):
        os.environ['CRDS_PATH'] = os.environ['CRDS_CACHE_TYPE']
print('CRDS cache location: {}'.format(os.environ['CRDS_PATH']))

CRDS cache location: /grp/crds/cache


## Set Up Import Statements

In [3]:
from astropy.io import fits, ascii
import pytest
import numpy as np
import numpy.ma as ma

import jwst
from jwst.pipeline import Detector1Pipeline, Image2Pipeline
from jwst.datamodels import RampModel, ImageModel, dqflags
from jwst.pipeline import calwebb_image2
from jwst.pipeline import calwebb_spec2

from ci_watson.artifactory_helpers import get_bigdata
import crds
import os
import sys

In [4]:
# Specify CRDS locations and pmap
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
os.environ['TEST_BIGDATA']='https://bytesalad.stsci.edu/artifactory/'

## Print pipeline version number 

In [5]:
jwst.__version__ 

'1.5.3'

## Read in data from Artifactory

In [6]:
file = get_bigdata('jwst_validation_notebooks',
                      'validation_data',
                      'calwebb_spec2',
                      'spec2_miri_test',
                      'det_image_seq1_MIRIFUSHORT_12LONGexp1_rate.fits')

## Define functions for validation and running pipeline

In [7]:
print('[*]', file=open('stpipe-log.cfg', "w"))
print('handler = file:pipeline.log', file=open('stpipe-log.cfg', "a"))
print('level = INFO', file=open('stpipe-log.cfg', "a"))

In [8]:
def run_spec2(im):
    """Creates and runs an instance of the JWST spec2 pipeline. """
    
    #only assign_wcs and srctype are run
    spec2 = calwebb_spec2.Spec2Pipeline()
    spec2.bkg_subtract.skip = True
    spec2.assign_wcs.skip = False
    spec2.imprint_subtract.skip = True
    spec2.msa_flagging.skip = True
    spec2.extract_2d.skip = True
    spec2.wavecorr.skip = True
    spec2.srctype.skip = False
    spec2.straylight.skip = True
    spec2.fringe.skip = True
    spec2.pathloss.skip = True
    spec2.barshadow.skip = True
    spec2.photom.skip = True
    spec2.resample_spec.skip = True
    spec2.cube_build.skip = True
    spec2.extract_1d.skip = True
    spec2.flat_field.skip = True
    spec2.save_results = True
        
    spec2.run(im)

In [9]:
def test_src_type(input_file, im1):
    """Passes data to spec2 pipeline and records the source type keyword of the output"""
    
    #run through spec2
    run_spec2(im1)
    
    #get the output file name
    cal_file = input_file.replace('rate.fits', 'cal.fits')
    
    #create an image model of the output and return the source type keyword
    im2 = ImageModel(cal_file)
    
    # Get the header source type value as well
    hdu=fits.open(cal_file)
    
    return im2.meta.target.source_type,hdu[1].header['SRCTYPE']

## Create an image model and test source type keywords

This test passes if none of the assertions return an error

In [10]:
im = ImageModel(file)

In [11]:
# Test the default case where no external information which should return EXTENDED
im.meta.target.source_type_apt = None
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = None

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'EXTENDED')
assert(src_type_hdr == 'EXTENDED')

2022-06-20 21:59:23,861 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.


2022-06-20 21:59:23,863 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2022-06-20 21:59:23,865 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2022-06-20 21:59:23,867 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.


2022-06-20 21:59:23,868 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.


...

2022-06-20 21:59:29,782 - stpipe.Spec2Pipeline - INFO - Finished processing product det_image_seq1_MIRIFUSHORT_12LONGexp1_rate


2022-06-20 21:59:29,783 - stpipe.Spec2Pipeline - INFO - Ending calwebb_spec2


2022-06-20 21:59:29,784 - stpipe.Spec2Pipeline - INFO - Results used CRDS context: jwst_0874.pmap


2022-06-20 21:59:30,709 - stpipe.Spec2Pipeline - INFO - Saved model in det_image_seq1_MIRIFUSHORT_12LONGexp1_cal.fits


2022-06-20 21:59:30,711 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline done


In [12]:
# Test the case where APT specified 'POINT'
im.meta.target.source_type_apt = 'POINT'
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = None

im.meta.target.source_type_apt = 'POINT'
    
src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'POINT')
assert(src_type_hdr == 'POINT')

2022-06-20 21:59:32,163 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.


2022-06-20 21:59:32,165 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2022-06-20 21:59:32,167 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2022-06-20 21:59:32,169 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.


2022-06-20 21:59:32,171 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.


...

2022-06-20 21:59:37,785 - stpipe.Spec2Pipeline - INFO - Finished processing product det_image_seq1_MIRIFUSHORT_12LONGexp1_rate


2022-06-20 21:59:37,786 - stpipe.Spec2Pipeline - INFO - Ending calwebb_spec2


2022-06-20 21:59:37,787 - stpipe.Spec2Pipeline - INFO - Results used CRDS context: jwst_0874.pmap


2022-06-20 21:59:38,695 - stpipe.Spec2Pipeline - INFO - Saved model in det_image_seq1_MIRIFUSHORT_12LONGexp1_cal.fits


2022-06-20 21:59:38,696 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline done


In [13]:
# Test the case where APT specified 'EXTENDED'
im.meta.target.source_type_apt = 'EXTENDED'
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = None

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'EXTENDED')
assert(src_type_hdr == 'EXTENDED')

2022-06-20 21:59:40,138 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.


2022-06-20 21:59:40,141 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2022-06-20 21:59:40,143 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2022-06-20 21:59:40,145 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.


2022-06-20 21:59:40,146 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.


...

2022-06-20 21:59:45,910 - stpipe.Spec2Pipeline - INFO - Finished processing product det_image_seq1_MIRIFUSHORT_12LONGexp1_rate


2022-06-20 21:59:45,910 - stpipe.Spec2Pipeline - INFO - Ending calwebb_spec2


2022-06-20 21:59:45,911 - stpipe.Spec2Pipeline - INFO - Results used CRDS context: jwst_0874.pmap


2022-06-20 21:59:46,862 - stpipe.Spec2Pipeline - INFO - Saved model in det_image_seq1_MIRIFUSHORT_12LONGexp1_cal.fits


2022-06-20 21:59:46,864 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline done


In [14]:
# If the background target keyword is set, it should always returns EXTENDED
# regardless of what APT said

im.meta.target.source_type_apt = 'POINT'
im.meta.observation.bkgdtarg = True

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'EXTENDED')
assert(src_type_hdr == 'EXTENDED')

2022-06-20 21:59:48,358 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.


2022-06-20 21:59:48,360 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2022-06-20 21:59:48,362 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2022-06-20 21:59:48,363 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.


2022-06-20 21:59:48,364 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.


...

2022-06-20 21:59:54,204 - stpipe.Spec2Pipeline - INFO - Finished processing product det_image_seq1_MIRIFUSHORT_12LONGexp1_rate


2022-06-20 21:59:54,205 - stpipe.Spec2Pipeline - INFO - Ending calwebb_spec2


2022-06-20 21:59:54,205 - stpipe.Spec2Pipeline - INFO - Results used CRDS context: jwst_0874.pmap


2022-06-20 21:59:55,139 - stpipe.Spec2Pipeline - INFO - Saved model in det_image_seq1_MIRIFUSHORT_12LONGexp1_cal.fits


2022-06-20 21:59:55,140 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline done


In [15]:
# If no information from APT, but dither keywords suggest a point source it should
# return POINT

im.meta.target.source_type_apt = None
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = 'POINT-SOURCE'

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'POINT')
assert(src_type_hdr == 'POINT')

2022-06-20 21:59:56,640 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.


2022-06-20 21:59:56,642 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2022-06-20 21:59:56,643 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2022-06-20 21:59:56,645 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.


2022-06-20 21:59:56,646 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.


...

2022-06-20 22:00:02,561 - stpipe.Spec2Pipeline - INFO - Finished processing product det_image_seq1_MIRIFUSHORT_12LONGexp1_rate


2022-06-20 22:00:02,562 - stpipe.Spec2Pipeline - INFO - Ending calwebb_spec2


2022-06-20 22:00:02,562 - stpipe.Spec2Pipeline - INFO - Results used CRDS context: jwst_0874.pmap


2022-06-20 22:00:03,483 - stpipe.Spec2Pipeline - INFO - Saved model in det_image_seq1_MIRIFUSHORT_12LONGexp1_cal.fits


2022-06-20 22:00:03,484 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline done


In [16]:
# If no information from APT, but dither keywords suggest an extended source it should
# return EXTENDED

im.meta.target.source_type_apt = None
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = 'EXTENDED-SOURCE'

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'EXTENDED')
assert(src_type_hdr == 'EXTENDED')

2022-06-20 22:00:04,989 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.


2022-06-20 22:00:04,992 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2022-06-20 22:00:04,993 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2022-06-20 22:00:04,995 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.


2022-06-20 22:00:04,996 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.


...

2022-06-20 22:00:11,019 - stpipe.Spec2Pipeline - INFO - Finished processing product det_image_seq1_MIRIFUSHORT_12LONGexp1_rate


2022-06-20 22:00:11,020 - stpipe.Spec2Pipeline - INFO - Ending calwebb_spec2


2022-06-20 22:00:11,021 - stpipe.Spec2Pipeline - INFO - Results used CRDS context: jwst_0874.pmap


2022-06-20 22:00:11,984 - stpipe.Spec2Pipeline - INFO - Saved model in det_image_seq1_MIRIFUSHORT_12LONGexp1_cal.fits


2022-06-20 22:00:11,986 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline done


In [17]:
# If the observation is a TSO, return POINT

im.meta.observation.bkgdtarg = False
im.meta.visit.tsovisit = True
im.meta.exposure.nints = 3

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'POINT')
assert(src_type_hdr == 'POINT')

2022-06-20 22:00:13,627 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.


2022-06-20 22:00:13,630 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2022-06-20 22:00:13,632 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2022-06-20 22:00:13,633 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.


2022-06-20 22:00:13,635 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.


...

2022-06-20 22:00:19,927 - stpipe.Spec2Pipeline - INFO - Finished processing product det_image_seq1_MIRIFUSHORT_12LONGexp1_rate


2022-06-20 22:00:19,927 - stpipe.Spec2Pipeline - INFO - Ending calwebb_spec2


2022-06-20 22:00:19,928 - stpipe.Spec2Pipeline - INFO - Results used CRDS context: jwst_0874.pmap


2022-06-20 22:00:20,855 - stpipe.Spec2Pipeline - INFO - Saved model in det_image_seq1_MIRIFUSHORT_12LONGexp1_cal.fits


2022-06-20 22:00:20,857 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline done


In [18]:
print('No problems found.')

No problems found.
